Redes Neuronales para Lenguaje Natural, 2025

---
# Laboratorio 2

En este laboratorio construiremos un sistema de Question Answering (QA) utilizando el método de Retrieval-Augmented Generation (RAG), que implica el uso de un paso de recuperación de información y un paso de generación de respuesta con LLM.

**Entrega: 18/11**

**Se debe entregar un archivo zip que contenga:**
* Este notebook de Python (.ipynb) completo.
* Los documentos obtenidos y utilizados como fuentes de información según se explica en la parte 1 (opcionalmente se puede entregar un archivo CSV con los textos de cada documento).
* Archivo CSV con el conjunto de preguntas y respuestas como se explica en la parte 5.

**No olvidar mantener todas las salidas de cada región de código en el notebook!**

---



## Parte 0


### Instalación bibliotecas

In [1]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  COLAB = True
else:
  print('Not running on CoLab')
  COLAB = False

#@title Estilo de salida de colab
from IPython.display import HTML, display, clear_output
if COLAB:
    pre_run_cell_fn = lambda: display(HTML('''<style> pre {white-space: pre-wrap;}</style>'''))
    get_ipython().events.register('pre_run_cell', pre_run_cell_fn)

import sys
!{sys.executable} -m pip install transformers -U
!{sys.executable} -m pip install bitsandbytes
!{sys.executable} -m pip install accelerate
!{sys.executable} -m pip install sentence-transformers
!{sys.executable} -m pip install evaluate
!{sys.executable} -m pip install bert_score
!{sys.executable} -m pip install google-genai
!{sys.executable} -m pip install pymupdf4llm
!{sys.executable} -m pip install -qU langchain-text-splitters

clear_output()

### Creación dataset preguntas y respuestas

In [9]:
import os
import csv
TEST_DATASET = "testset.csv"

if not os.path.exists(TEST_DATASET):
    with open(TEST_DATASET, "w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["question", "answer", "source"])

        # No relacionadas con el dominio
        respuesta_para_pregunta_fuera_de_tema = "Lo siento, no cuento con información para responder esa pregunta."
        writer.writerow(["¿Dónde nace el río Uruguay?", respuesta_para_pregunta_fuera_de_tema, "N/A"])
        writer.writerow(["¿En qué año se firmó el tratado de Tordesillas?", respuesta_para_pregunta_fuera_de_tema, "N/A"])
        writer.writerow(["¿Cuál es el presidente actual de Chile?", respuesta_para_pregunta_fuera_de_tema, "N/A"])
        writer.writerow(["¿Cuál es la capital de Francia?", respuesta_para_pregunta_fuera_de_tema, "N/A"])
        writer.writerow(["¿Quién ganó la Copa del Mundo de la FIFA 2022?", respuesta_para_pregunta_fuera_de_tema, "N/A"])
        writer.writerow(["¿Cuál es la fórmula química del agua?", respuesta_para_pregunta_fuera_de_tema, "N/A"])

        # Necesitan información de un chunk
        writer.writerow(["¿Es la Biblioteca Nacional más antigua que la república de Uruguay?", "Sí, la Biblioteca Nacional es más antigua que la propia república.", "Presentacion.pdf"])
        writer.writerow(["¿En qué año fue nombrado Francisco Acuña de Figueroa director de la Biblioteca Nacional?", "Fue nombrado a mediados de 1840.", "Acuña de Figueroa.pdf"])
        writer.writerow(["¿Qué director de la Biblioteca Nacional es también considerado el primer poeta de la patria?", "Francisco Acuña de Figueroa.", "Acuña de Figueroa.pdf"])
        writer.writerow(["¿Qué botánico francés visitó la biblioteca cerrada hacia 1820 y estimó su colección?", "Auguste de Saint-Hilaire, quien estimó que el número de libros era de aproximadamente dos mil.", "Clausura, expolios, intentos de reapertura.pdf"])
        writer.writerow(["¿Quién fue el arquitecto que ganó el concurso de 1937 para el actual edificio de la Biblioteca Nacional?", "El arquitecto Luis Crespi.", "El edificio de la Biblioteca Nacional.pdf"])
        writer.writerow(["¿Qué director propuso en 1873 la ""Creación de una Nueva Biblioteca Nacional"" mediante una suscripción popular?", "Juan Antonio Tavolara.", "La Nueva Biblioteca Nacional.pdf"])
        writer.writerow(["¿Qué relación tuvieron Dámaso Antonio Larrañaga y Acuña de Figueroa con la Biblioteca Nacional?", "Dámaso Antonio Larrañaga fue el protagonista del acto fundacional de la biblioteca. Francisco Acuña de Figueroa fue director de la biblioteca durante siete años, en los complejos tiempos del Sitio Grande de Montevideo.", "Presentacion.pdf;Acuña de Figueroa.pdf"])
        writer.writerow(["¿Qué dos directores de la Biblioteca Nacional tuvieron gestiones criticadas o problemáticas?", "La gestión de Francisco Acuña de Figueroa fue polémica por su ""camaleonismo político"" y su fidelidad a Rivera, que llevó a su destitución. La gestión de Juan Antonio Tavolara fue criticada por rematar obras consideradas ""inservibles"" a vil precio y Fernández Saldaña la consideró ""poco brillante"".", "Acuña de Figueroa.pdf;La Nueva Biblioteca Nacional.pdf"])


        # Necesitan información de más de un chunk
        # <<...la construcción de la sede actual...>> <<...se trasladó...>> <<..el alquiler ... que la biblioteca ocupaba desde 1894,31 en la calle Florida n.º 93.. >> <<...no es solo un edificio ... en 18 de julio..>>
        writer.writerow(["¿La Biblioteca Nacional siempre estuvo en su lugar actual?", "No, la sede de la Biblioteca Nacional ha cambiado varias veces a lo largo de la historia.", "Presentacion.pdf, La Nueva Biblioteca Nacional.pdf, El edificio de la Biblioteca Nacional.pdf"])

        # <<...actualmente cuenta con 850.000 volúmenes...>> <<..entre 1916 y 1933 se pasó de 59.552 a 132.442 volúmenes...>>
        writer.writerow(["¿En que año la Biblioteca Nacional contó con más volúmenes?", "En 2021, al alcanzar los 850.000 volúmenes", "La Nueva Biblioteca Nacional.pdf, El edificio de la Biblioteca Nacional.pdf"])

        # <<..fue director por 7 años...>> <<..cuando a mediados de 1840... decidió conferirle el cargo de director...>>
        writer.writerow(["¿En que año terminó el mandato de Francisco Acuña de Figueroa como director de la Biblioteca Nacional?", "En 1847", "Acuña de Figueroa.pdf"])


## Parte 1: Procesamiento de los documentos

En esta parte, cada grupo deberá construir y procesar su conjunto de documentos. Esto consiste de los siguientes pasos:

* Elegir un tema dentro de un dominio específico sobre el que trabajar.
* Obtener al menos 5 documentos en español que contengan información sobre el tema elegido.
* Procesar cada documento para extraer el texto del formato original a un string en Python (por ejemplo, extraer el texto de un PDF).

El resultado de esta parte debe ser una lista cargada en memoria que contenga el texto (string) de cada uno de los documentos elegidos.

**Sugerencias:**
* Se recomienda utilizar artículos de wikipedia para simplificar la etapa de extracción del texto (ver la librería [wikipedia-api](https://github.com/martin-majlis/Wikipedia-API/)).
* Opcionalmente puede utilizar documentos PDF, páginas web u otros formatos. En estos casos se sugiere:
  * Utilizar la librería PyPDF2 para procesar documentos PDF.
  * Utilizar la librería LangChain para procesar páginas web, en particular la clase Html2TextTransformer, que convierte HTML a Markdown ([ejemplo de uso](https://python.langchain.com/v0.2/docs/integrations/document_transformers/html2text/)).
* Puede ser conveniente guardar el resultado del procesamiento de los documentos en un archivo CSV (donde cada fila corresponde al texto de un documento) para no tener que repetir este proceso cada vez que se ejecuta el notebook, y en su lugar cargar el archivo CSV.

In [10]:
import pymupdf4llm
import csv

CORPUS_PATH = "corpus.csv"

def extractTextFromPdf(file_name):
    markdown = pymupdf4llm.to_markdown(file_name)
    return markdown

def writeIntoCsv(file_name, text):
    with open(CORPUS_PATH, "a", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([file_name, text])

def addFileToCsv(file_name):
    text = extractTextFromPdf(file_name)
    writeIntoCsv(file_name, text)

def createCsv():
    with open(CORPUS_PATH, "w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["file_name", "text"])


In [13]:

if not os.path.exists(CORPUS_PATH):
  files = ["Presentacion.pdf", "Acunha de Figueroa.pdf", "Clausura, expolios, intentos de reapertura.pdf", "El edificio de la Biblioteca Nacional.pdf", "La Nueva Biblioteca Nacional.pdf"]

  createCsv()
  for file_name in files:
    addFileToCsv(file_name)

Los textos resultantes deben estar almacenados en la variable `documents`:

In [15]:
import pandas as pd
documents = pd.read_csv(CORPUS_PATH, header = 0, index_col='file_name')

In [16]:
print(documents)

                                                                                             text
file_name                                                                                        
Presentacion.pdf                                Revista de la Biblioteca Nacional. 17, 5, 2021...
Acunha de Figueroa.pdf                          Revista de la Biblioteca Nacional. 17, 71-90, ...
Clausura, expolios, intentos de reapertura.pdf  Revista de la Biblioteca Nacional. 17, 53-69, ...
El edificio de la Biblioteca Nacional.pdf       Revista de la Biblioteca Nacional. 17, 225-265...
La Nueva Biblioteca Nacional.pdf                Revista de la Biblioteca Nacional. 17, 109-117...


## Parte 2: Chunking

Una vez que se obtiene el texto de cada documento, se debe realizar la etapa de _chunking_. Esta etapa consiste en dividir cada texto en segmentos más chicos a los que llamamos _chunks_.

Realizar la etapa de _chunking_ de forma automática utilizando un método simple que permita obtener _chunks_ de un largo aproximado de 500 caracteres.

Puede probar con dividir a nivel de caracteres, palabras o incluso párrafos, teniendo en cuenta que el largo de cada _chunk_ no debería exceder demasiado los 500 caracteres.

**Sugerencias:**
* Puede utilizar los splitters disponibles en LangChain ([documentación](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)) como RecursiveCharacterTextSplitter, aunque no es obligatorio y también es correcto hacer una implementación propia.
* Tener en cuenta que esta etapa es crucial en el resultado final. Cuanto más contextualizados queden los *chunks*, mejor será el rendimiento de la etapa de recuperación de información. Es conveniente minimizar la división de palabras (o párrafos) por la mitad.

#### Definición de splitters

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter

from enum import StrEnum

class SplitterType(StrEnum):
    RECURSIVE = "Recursive"
    MARKDOWN = "Markdown"

## Recursive Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=200, # Con overlaps más chicos el splitter ignora el parámetro en la nueva versión de langchain
    length_function=len,
    is_separator_regex=False
)

def splitRecursivo(text):
  chunks = text_splitter.split_text(text)
  return chunks

## Markdown Splitter
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on,
    return_each_line=True
)

def splitMarkdown(text):
  markdown_chunks = markdown_splitter.split_text(text)
  chunks = text_splitter.split_documents(markdown_chunks)
  return chunks

In [18]:
# incorpora la jerarquía de headers al texto base en el caso del Markdown Splitter
def agregar_titulos(chunks):
    for chunk in chunks:
      header_string = "\n".join(chunk.metadata.values())
      final_string = f"{header_string}\n{chunk.page_content}"
      chunk.page_content = final_string
    return chunks

In [19]:
def chunk_text(text, splitter=SplitterType.RECURSIVE):

  if splitter == SplitterType.RECURSIVE:
    chunks = splitRecursivo(text)
  elif splitter == SplitterType.MARKDOWN:
    chunks = splitMarkdown(text)
    chunks = agregar_titulos(chunks)

  return chunks # Lista de strings con los chunks del texto

In [20]:
def chunk_documents(chunk_list, splitter=SplitterType.RECURSIVE):
  for document_text in documents['text']:
      chunk_list += chunk_text(document_text,splitter=splitter)
  if splitter == SplitterType.RECURSIVE:
        print("Documents chunked using recursive splitter")
  elif splitter == SplitterType.MARKDOWN:
        print("Documents chunked using markdown splitter")
  return chunk_list

In [21]:
recursive_chunks = []
recursive_chunks = chunk_documents(recursive_chunks, splitter=SplitterType.RECURSIVE)
markdown_chunks = []
markdown_chunks = chunk_documents(markdown_chunks, splitter=SplitterType.MARKDOWN)

Documents chunked using recursive splitter
Documents chunked using markdown splitter


## Parte 3: Recuperación de información

En esta parte vamos a implementar el método de recuperación de información que nos permitirá obtener los _chunks_ más relevantes para la pregunta.

En primer lugar, cargamos el modelo Bi-Encoder que utilizaremos para generar los embeddings utilizando la librería sentence_transformers.

Se utiliza el modelo multilingüe [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large), fine-tuning del modelo `xlm-roberta-large` para la tarea de generación de sentence embeddings.

Se pueden explorar otros modelos Bi-Encoder, e incluso modelos Cross-Encoder o del tipo ColBERT. En HuggingFace se puede consultar el siguiente [leaderboard](https://huggingface.co/spaces/mteb/leaderboard) que compara varios modelos de este tipo en diferentes tareas.

In [23]:
from sentence_transformers import SentenceTransformer

model_emb = SentenceTransformer("intfloat/multilingual-e5-large")
clear_output()

A continuación se debe generar las representaciones vectoriales para todos los _chunks_ ([ejemplo de uso](https://huggingface.co/intfloat/multilingual-e5-large#support-for-sentence-transformers)).

**Observación:** El modelo que estamos usando espera que los _chunks_ comiencen con el prefijo `passage: ` por lo que será necesario agregarlo al inicio de todos los _chunks_.

In [24]:
def prepare_chunks_for_embedding(chunks):
    prepared_chunks = []
    i = 0
    for chunk in chunks:
        # Si es un objeto Document (del markdown splitter), extraer el contenido
        if hasattr(chunk, 'page_content'):
            text = chunk.page_content
        else:
            # Si es un string simple
            text = chunk

        prepared_chunks.append(f"passage: {text}")

    return prepared_chunks

chunks_with_prefix_recursive = prepare_chunks_for_embedding(recursive_chunks)
chunks_with_prefix_markdown = prepare_chunks_for_embedding(markdown_chunks)

print(f"Generando embeddings para {len(chunks_with_prefix_recursive)} chunks recursivos...")
chunk_embeddings_recursive = model_emb.encode(chunks_with_prefix_recursive, normalize_embeddings=True)

print(f"Generando embeddings para {len(chunks_with_prefix_markdown)} chunks markdown...")
chunk_embeddings_markdown = model_emb.encode(chunks_with_prefix_markdown, normalize_embeddings=True)

print(f"Embeddings recursivos generados. Dimensión: {chunk_embeddings_recursive.shape}")
print(f"Embeddings markdown generados. Dimensión: {chunk_embeddings_markdown.shape}")

Generando embeddings para 503 chunks recursivos...
Generando embeddings para 742 chunks markdown...
Embeddings recursivos generados. Dimensión: (503, 1024)
Embeddings markdown generados. Dimensión: (742, 1024)


Por último, se debe implementar el algoritmo de búsqueda de los embeddings más cercanos para un embedding dado.

**Sugerencias:**
* Utilizar la clase NearestNeighbors de sklearn ([documentación](https://scikit-learn.org/dev/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)).

In [25]:
from sklearn.neighbors import NearestNeighbors

nn_model_recursive = NearestNeighbors(
    n_neighbors=5,
    metric='cosine',
    algorithm='brute'
)

nn_model_recursive.fit(chunk_embeddings_recursive)

print("Modelo de vecinos más cercanos recursivo entrenado")

nn_model_markdown = NearestNeighbors(
    n_neighbors=5,
    metric='cosine',
    algorithm='brute'
)

nn_model_markdown.fit(chunk_embeddings_markdown)

print("Modelo de vecinos más cercanos markdown entrenado")

Modelo de vecinos más cercanos recursivo entrenado
Modelo de vecinos más cercanos markdown entrenado


In [26]:
def retrieve_chunks(all_chunks, nn_model, query, top_k=3):
    """
    Returns:
        Una tupla (chunks_recuperados, distancias, indices)
        - chunks_recuperados: Lista de textos de los chunks más relevantes
        - distancias: Distancias coseno a cada chunk
        - indices: Índices de los chunks en la lista original
    """
    query_with_prefix = f"query: {query}"

    query_embedding = model_emb.encode([query_with_prefix], normalize_embeddings=True)

    distances, indices = nn_model.kneighbors(query_embedding, n_neighbors=top_k)

    # Extraer los chunks correspondientes
    chunks_recuperados = []
    for idx in indices[0]:
        chunk = all_chunks[idx]
        # Manejar tanto strings como objetos Document
        if hasattr(chunk, 'page_content'):
            chunks_recuperados.append(chunk.page_content)
        else:
            chunks_recuperados.append(chunk)

    return chunks_recuperados, distances[0], indices[0]

## Parte 4: Generación de respuestas

### Configuración de LLM

En esta parte, implementaremos un wrapper flexible que permite experimentar con diferentes modelos de lenguaje:

1. **Llama 3.1** (modelo abierto): Utilizaremos el modelo Meta-Llama-3.1-8B-Instruct a través de HuggingFace.
2. **Gemini 2.0 Flash** (modelo cerrado): Utilizaremos la API de Google Gemini.

Para **Llama 3.1**, es necesario:
- Crearse una cuenta de HuggingFace (https://huggingface.co/)
- Aceptar los términos para usar el modelo en HuggingFace: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct
- Crear un token de HuggingFace con permiso de lectura: https://huggingface.co/settings/tokens

Para **Gemini**, es necesario:
- Obtener una API key de Google AI Studio: https://aistudio.google.com/app/apikey

In [28]:
# Ejecutar para conectarse a HuggingFace (solo necesario para Llama 3.1)
from huggingface_hub import notebook_login, login

if COLAB:
    notebook_login()
else:
    token = ''
    login(token=token)


### Definición del Wrapper de LLM

A continuación se define una clase abstracta que permite intercambiar fácilmente entre diferentes modelos de lenguaje.

In [29]:
from abc import ABC, abstractmethod
from typing import Optional
import os
from getpass import getpass

class LLMWrapper(ABC):
    """Clase base abstracta para wrappers de modelos de lenguaje."""

    @abstractmethod
    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        pass

    @abstractmethod
    def get_model_name(self) -> str:
        pass


class LlamaWrapper(LLMWrapper):
    """Wrapper para el modelo Llama 3.1 de Meta vía HuggingFace."""

    def __init__(self):
        from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
        import torch

        print("Inicializando Llama 3.1...")

        self.tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct")


        if COLAB:
            # Configuración de cuantización a 4 bits (para mejorar eficiencia)
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16)


            self.model = AutoModelForCausalLM.from_pretrained(
                "meta-llama/Meta-Llama-3.1-8B-Instruct",
                quantization_config=bnb_config,
                device_map="auto")
        else:
            self.model = AutoModelForCausalLM.from_pretrained(
                "meta-llama/Meta-Llama-3.1-8B-Instruct",
                device_map="auto")

        print("Llama 3.1 inicializado correctamente")

    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        from transformers import GenerationConfig, pipeline

        # Configuración de temperatura
        generation_config = GenerationConfig(
            temperature=temperature if temperature > 0 else None,
            do_sample=temperature > 0)

        # Inicializar pipeline para generación de texto
        pipe = pipeline(
            "text-generation",
            model=self.model,
            config=generation_config,
            tokenizer=self.tokenizer,
            pad_token_id=self.tokenizer.eos_token_id)

        # Generar texto
        output = pipe(
            prompt,
            return_full_text=False,
            max_new_tokens=max_tokens)

        return output[0]['generated_text']

    def get_model_name(self) -> str:
        return "Llama-3.1-8B-Instruct"


class GeminiWrapper(LLMWrapper):
    """Wrapper para el modelo Gemini 2.0 Flash de Google."""

    def __init__(self, api_key: Optional[str] = None):
        from google import genai

        print("Inicializando Gemini 1.5 Flash...")

        if api_key is None:
            api_key = os.environ.get("GEMINI_API_KEY")

        if api_key is None:
            print("No se encontró la API key de Gemini en las variables de entorno.")
            api_key = getpass("Por favor, ingrese su API key de Gemini: ")
            # Guardar en variables de entorno para esta sesión
            os.environ["GEMINI_API_KEY"] = api_key

        self.client = genai.Client(api_key=api_key)

        print("Gemini 2.0 Flash inicializado correctamente")

    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        from google.genai import types

        config = types.GenerateContentConfig(
            temperature=temperature if temperature > 0 else 0.0,
            max_output_tokens=max_tokens)

        # Generar respuesta
        response = self.client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt,
            config=config)

        return response.text

    def get_model_name(self) -> str:
        return "Gemini-2.0-Flash"

### Instanciar modelos

Seleccionar qué modelo(s) se va a inicializar. Se puede inicializar ambos para facilitar la experimentación posterior.

In [ ]:
llama_model = LlamaWrapper()
gemini_model = GeminiWrapper()

clear_output()


### Función auxiliar para generación de respuestas

Esta función utiliza el modelo activo seleccionado anteriormente.

In [31]:
def get_response(
    prompt: str,
    model: LLMWrapper = None,
    temp: float = 0.0,
    max_tok: int = 500
) -> str:
    if model is None:
        model = active_model

    return model.generate(prompt, temperature=temp, max_tokens=max_tok)

### Crear prompt y generar respuesta

Escribir la función `create_prompt(question, use_chat_template=True)` que dada una pregunta, genere la prompt que se utilizará para generar la respuesta. Tener en cuenta que se debe realizar la búsqueda semántica de los _chunks_ más cercanos a la pregunta utilizando lo implementado en la parte 3.

**Observación:** Al igual que para los _chunks_, el modelo Bi-Encoder espera que la pregunta comience con un prefijo especial: `query: ` por lo que será necesario agregarlo al inicio de la pregunta para generar el embedding.

**Sugerencias:**
* Puede probar con distintas cantidades de _chunks_ recuperados, pero se sugiere comenzar con 3. Tener en cuenta que más _chunks_ recuperados y agregados en la prompt implica mayor uso de memoria en inferencia.
* El parámetro `use_chat_template` permite controlar si se aplica el template de chat de Llama (necesario para Llama 3.1, opcional para Gemini). Para Llama usar `True`, para Gemini se puede probar con `True` o `False` según el formato que se prefiera.

In [38]:
def create_prompt(question, active_model, use_chat_template=True, model_for_template=None, top_k=3, chunk_list=recursive_chunks, nn_model=nn_model_recursive):
    """
    Crea el prompt para el modelo de lenguaje incluyendo contexto recuperado.

    Args:
        question: La pregunta del usuario
        use_chat_template: Si True, aplica el template de chat de Llama
        model_for_template: Modelo del cual usar el tokenizer (solo para Llama)
        top_k: Número de chunks a recuperar (por defecto 3)

    Returns:
        El prompt formateado
    """

    retrieved_chunks, _, _ = retrieve_chunks(chunk_list, nn_model, question, top_k=top_k)

    context = "\n\n".join(retrieved_chunks)

    system_message = """Eres un asistente experto en responder preguntas basándote únicamente en el contexto proporcionado.

Instrucciones:
- Si la información para responder la pregunta está en el contexto, proporciona una respuesta clara y concisa.
- Si la información NO está en el contexto, responde: "Lo siento, no cuento con información para responder esa pregunta."
- No inventes información que no esté en el contexto.
- Responde en español."""

    user_message = f"""Contexto:
{context}

Pregunta: {question}"""

    if use_chat_template:
        if model_for_template is None and isinstance(active_model, LlamaWrapper):
            model_for_template = active_model

        if isinstance(model_for_template, LlamaWrapper):
            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ]
            prompt = model_for_template.tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
        else:
            prompt = f"{system_message}\n\n{user_message}"
    else:
        prompt = f"{system_message}\n\n{user_message}"

    return prompt

### Análisis de chunks obtenidos para ciertas preguntas



In [74]:
question = "¿Quién fue el arquitecto que ganó el concurso de 1937 para el actual edificio de la Biblioteca Nacional?"
retrieved_chunks, _, _ = retrieve_chunks(recursive_chunks, nn_model_recursive, question, top_k=3)

context = "\n\n".join(retrieved_chunks)
print("RecursiveSplitter")
print(context)
print('\n\n\n')

retrieved_chunks, _, _ = retrieve_chunks(markdown_chunks, nn_model_markdown, question, top_k=3)

context = "\n\n".join(retrieved_chunks)
print("MarkdownSplitter")
print(context)

RecursiveSplitter
propia idea de lo que significa una biblioteca nacional y cómo esto
se tradujo en arquitectura. Se examinan así proyectos anteriores para
la sede de la biblioteca, las vicisitudes del concurso de 1937, las modificaciones hechas por su autor, el arquitecto Luis Crespi, en 1942
y las valoraciones contemporáneas y posteriores a su inauguración.

La idea de vincular los libros y la lectura, entendidos como
motores del «perfeccionamiento espiritual», a una dimensión
sagrada, no era una novedad en octubre de 1937, cuando el arquitecto Luis Crespi [3] entregó el anteproyecto que, a la postre,
sería el germen del actual edificio de la Biblioteca Nacional (BN).
Existían numerosos casos de bibliotecas, tanto históricas como
contemporáneas, que aludían, de diversas maneras, a los templos

**El concurso (1937)**


Quince anteproyectos se presentaron al concurso para Biblioteca
Nacional y Museo Histórico Nacional (MHN), reservado, como era
usual, a arquitectos nacionales. Este dob

In [75]:
question = "¿En que año la Biblioteca Nacional contó con más volúmenes?"
retrieved_chunks, _, _ = retrieve_chunks(recursive_chunks, nn_model_recursive, question, top_k=3)

context = "\n\n".join(retrieved_chunks)
print("RecursiveSplitter")
print(context)
print('\n\n\n')

retrieved_chunks, _, _ = retrieve_chunks(markdown_chunks, nn_model_markdown, question, top_k=3)

context = "\n\n".join(retrieved_chunks)
print("MarkdownSplitter")
print(context)

RecursiveSplitter
En 1870, según el inventario reproducido por Adolfo Vaillant en
su «Anuario y Almanaque», la Biblioteca Nacional contaba 3.653
volúmenes, 970 folletos, 337 tomos de diarios encuadernados y 179
de diarios sin encuadernar, « _cifras miserables que denuncian el estado_
_de abandono o de crisis del establecimiento_ » [15] a juicio de Acevedo.

39. Un informe indica que el volumen del acervo casi se triplicó entre 1916 y 1933. Se pasó
de 37.398 obras en 59552 volúmenes y folletos a un total de 91.767 obras en 132.442 volúmenes, folletos y hojas sueltas, 235 grabados, 271 mapas y planos y 617 piezas musicales.
En 1932 concurrieron a la BN 58.966 lectores. AABN. Libro Asuntos, 1933, vol. 1. Asunto
7292, folio 46. En 1935, la cantidad de lectores había aumentado a 75.982. AABN. Libro

En 1867, la Biblioteca Nacional se trasladó a la planta alta de la
Casa de Correos (inaugurada en ese mismo año), lo que dio lugar
a que su director mandara a remate público obras consideradas «

In [76]:
question = '¿En que año terminó el mandato de Francisco Acuña de Figueroa como director de la Biblioteca Nacional?'
retrieved_chunks, _, _ = retrieve_chunks(recursive_chunks, nn_model_recursive, question, top_k=3)

context = "\n\n".join(retrieved_chunks)
print("RecursiveSplitter")
print(context)
print('\n\n\n')

retrieved_chunks, _, _ = retrieve_chunks(markdown_chunks, nn_model_markdown, question, top_k=3)

context = "\n\n".join(retrieved_chunks)
print("MarkdownSplitter")
print(context)

RecursiveSplitter
Cuando a mediados de 1840 el gobierno oriental presidido por
el general Fructuoso Rivera decidió conferirle el cargo de director de
la Biblioteca Nacional, Francisco Acuña de Figueroa era no solo el
más importante poeta de la joven República, sino también, por lejos,
el más polémico. En la agitada década anterior, Acuña de Figueroa
había ganado toda la atención popular cuando en 1833 el presidente

Revista de la Biblioteca Nacional. 17, 71-90, 2021. ISSN 0797-9061


asamblea el 22 setiembre de 1847 a favor del destierro del caudillo.
Acuña de Figueroa demostró su fidelidad a Rivera al no participar de
la asamblea. Los colorados no riveristas tomaron apunte del hecho y
lo destituyeron de la dirección de la Biblioteca Nacional.

Luego de que dejó la dirección de la Biblioteca Nacional, Acuña
de Figueroa siguió escribiendo en mil formas diferentes, sobre mil
temas distintos, con estilo vivaz. Alabó a todos los gobernantes de
turno, hasta su muerte en 1862. Escribió Galli

## Parte 5: Evaluación
A continuación vamos a evaluar la solución construida. Para ello, se deben seguir los siguientes pasos:

* Construir un conjunto de evaluación de forma manual que contenga al menos 12 preguntas y respuestas con las siguientes características:
  * Al menos 3 preguntas deben necesitar información presente en más de un _chunk_ para ser respondidas correctamente.
  * Al menos 3 preguntas no deben estar relacionadas con el dominio, y su respuesta de referencia debe ser algo similar a: "Lo siento, no cuento con información para responder esa pregunta."
* El conjunto debe estar en un archivo CSV llamado testset.csv, con las columnas "question" y "answer".

Se deberá realizar al menos tres experimentos diferentes y evaluar sobre el mismo conjunto de test con la métrica BERTScore. Los experimentos deben variar en al menos uno de los siguientes elementos:
* Método de chunking
* Modelo (o método) de retrieval
* Modelo de generación (LLM)
* Método de prompting (se puede probar con few-shot, chain of thought, etc)
* Otros aspectos que considere relevantes a probar

A continuación se definen funciones auxiliares para la evaluación.


In [54]:
import evaluate
import numpy as np
import time
from tqdm.notebook import tqdm

def generate_predictions(
    questions,
    model=None,
    use_chat_template=True,
    use_delay=False,
    chunk_list=recursive_chunks,
    nn_model=nn_model_recursive
):
    if model is None:
        model = active_model

    predictions = []
    for question in tqdm(questions, desc=f"Generando con {model.get_model_name()}"):
        prompt = create_prompt(question, model, use_chat_template=use_chat_template, chunk_list=chunk_list, nn_model=nn_model)
        prediction = get_response(prompt, model=model)
        if use_delay:
            time.sleep(10)
        predictions.append(prediction)

    return predictions

def evaluate_predictions(predictions, references, experiment_name=""):
    """Evalúa predicciones usando BERTScore."""
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=predictions, references=references, lang='es')

    metrics = {
        'precision': np.array(results['precision']).mean(),
        'recall': np.array(results['recall']).mean(),
        'f1': np.array(results['f1']).mean()
    }

    if experiment_name:
        print(f"\n=== Resultados: {experiment_name} ===")
    print(f"BERTScore P:  {metrics['precision']:.3f}")
    print(f"BERTScore R:  {metrics['recall']:.3f}")
    print(f"BERTScore F1: {metrics['f1']:.3f}")

    return metrics

In [41]:
import pandas as pd

df = pd.read_csv("testset.csv")

questions = df["question"].tolist()
references = df["answer"].tolist()

Evalúe los experimentos realizados.

In [42]:
import json
from google.colab import files

In [ ]:
# Evaluar experimentos
# Almacenar resultados para comparación posterior
results_dict = {}

experimentos = [
    {
        "numero": 1,
        "nombre": "Exp1: Llama 3.1 con chunking recursivo",
        "modelo": llama_model,
        "chunking": recursive_chunks,
        "nn_model": nn_model_recursive
    },
    {
        "numero": 2,
        "nombre": "Exp2: Llama 3.1 con chunking markdown",
        "modelo": llama_model,
        "chunking": markdown_chunks,
        "nn_model": nn_model_markdown
    },
    {
        "numero": 3,
        "nombre": "Exp3: Gemini 2.0 Flash con chunking markdown",
        "modelo": gemini_model,
        "chunking": markdown_chunks,
        "nn_model": nn_model_markdown
    }
]

respuestas = []
evaluaciones = []
for experimento in experimentos:
    modelo = experimento["modelo"]
    chunking = experimento["chunking"]
    nearest_neighbors = experimento["nn_model"]

    respuesta = generate_predictions(questions, model=modelo, use_chat_template=True, chunk_list=chunking, nn_model=nearest_neighbors)
    respuestas.append({ f"experimento{experimento['numero']}": respuesta })
    evaluacion = evaluate_predictions(respuesta, references, experimento["nombre"])
    evaluaciones.append({ f"experimento{experimento['numero']}": evaluacion })

In [57]:
print(evaluaciones)

[{'experimento1': {'precision': np.float64(0.8302036348511191), 'recall': np.float64(0.8368074929012972), 'f1': np.float64(0.831717287792879)}}, {'experimento2': {'precision': np.float64(0.8580391091458938), 'recall': np.float64(0.8581515796044293), 'f1': np.float64(0.857164740562439)}}, {'experimento3': {'precision': np.float64(0.83458612946903), 'recall': np.float64(0.8652663721757776), 'f1': np.float64(0.8488758942660164)}}]


In [66]:
for respuesta in respuestas[0]['experimento1']:
  print(respuesta)

print('\n\n')

for respuesta in respuestas[1]['experimento2']:
  print(respuesta)

print('\n\n')


for respuesta in respuestas[2]['experimento3']:
  print(respuesta)

print('\n\n')




Lo siento, no cuento con información para responder esa pregunta.
Lo siento, no cuento con información para responder esa pregunta.
Lo siento, no cuento con información para responder esa pregunta.
Lo siento, no cuento con información para responder esa pregunta.
Lo siento, no cuento con información para responder esa pregunta.
Lo siento, no cuento con información para responder esa pregunta.
Sí, según el texto, la Biblioteca Nacional es más antigua que la república de Uruguay.
En el año 1840.
Francisco Acuña de Figueroa es considerado el primer poeta de la patria y también fue director de la Biblioteca Nacional.
Auguste de Saint-Hilaire.
Lo siento, no cuento con información para responder esa pregunta.
Lo siento, no cuento con información para responder esa pregunta.
Dámaso Larrañaga fundó la Biblioteca Nacional en 1816.
Los dos directores de la Biblioteca Nacional cuya gestión tuvo problemas o fue criticada, según el contexto proporcionado, fueron:

1. Tomás Vilardebó (en 1837): Fue 

### Reporte de resultados

Reportar los resultados obtenidos en los experimentos realizados completando la siguiente tabla:

| Exp | Descripción | P BERTScore | R BERTScore | F BERTScore |
|-----|-------------|-------------|-------------|-------------|
| 1 | Llama 3.1 RecursiveSplitter| 0.830| 0.837| 0.832 |
| 2 | Llama 3.1 MarkdownSplitter| 0.858| 0.858 |0.857 |
| 3 | Gemini 2.0 Flash MarkdownSplitter | 0.835| 0.865 | 0.849 |

Responda las siguientes preguntas:

1. Explique brevemente las diferencias en los experimentos realizados, ¿Qué aspectos se varió en el pipeline de RAG?

2. ¿Son consistentes los resultados obtenidos con lo que esperaba?

3. ¿Le parece que la métrica BERTScore está capturando correctamente las diferencias de los distintos experimentos realizados?

**Pregunta 1**

Se realizaron variaciones en el método de chunking y el modelo generativo. No se probaron variaciones en el modelo de embeddings (multilingual-e5-large) ni el algoritmo de recuperación de información (k-NN con k=5). Se buscó variar solo un aspecto a la vez para poder evaluar el efecto de éste sobre la performance dentro de lo posible (no tenemos control sobre el no-determinismo propio de los LLMs).
1. En el primer experimento se utilizó el splitter Recursivo de Langchain para realizar el chunking y Llama 3.1 como modelo generativo.
2. En el segundo experimento se utilizó el splitter Markdown de Langchain y luego nuevamente Llama 3.1 como modelo generativo.
3. Finalmente, en el último experimento se mantuvo el splitter Markdown de Langchain y se utilizó Gemini-2.0-Flash como modelo generativo.


**Pregunta 2**

Los resultados superaron las expectativas, principalmente para la capacidad de los modelos de embeddings de recuperar el contexto (los chunks) adecuados para cada pregunta. Llama la atención como con un pipeline simple se pueden obtener resultados relativamente buenos. Probablemente puedan ser mejorados con una mejor etapa de recuperación de información.

**Pregunta 3**


Para el problema planteado, las respuestas generadas por los modelos de los experimentos no tenían grandes diferencias entre sí, y la métrica BERTScore reflejó esta similitud.
No sé presentó una situación que resaltara ni las limitaciones de esta métrica, ni las diferencias entre los experimentos, probablemente por la forma estructurada de responder cuándo no sé contaba con información en el contexto.

**Evaluación Humana**

Además de las métricas utilizadas en el reporte de resultado, resulta de interés evaluar el dataset de preguntas de test de forma manual, para poder contrastar y visualizar los matices de las mismas.


| Exp | Descripción | Preguntas No Relacionadas | Preguntas (1 chunk) | Preguntas (2+ chunks) |
|-----|-------------|-------------|-------------|-------------|
| 1 | Llama 3.1 RecursiveSplitter| 6/6 | 5/8 * | 1/3 |
| 2 | Llama 3.1 MarkdownSplitter| 6/6 | 5/8 * | 0.5/3 * |
| 3 | Gemini 2.0 Flash MarkdownSplitter| 6/6 | 6.5/8 * | 0.5/3 * |

El peor desempeño se observó consistentemente en las preguntas para las cuáles se precisaba más de un chunk para poder responderlas.
Una de ellas, la referida a en qué año se contó con más volúmenes, implicaba cierto razonamiento (encontrar todas las menciones de volúmenes y años y elegir el mayor). Tenía cierta trampa porque en un chunk se compara la cantidad de volúmenes entre dos años, pero en otro chunk se menciona una cantidad de volúmenes mayor.
Al analizar los chunks recuperados para la misma, se observó que para ningún splitter se obtenían los adecuados. Esto probablemente se deba a que el chunk que contaba con la información de en que año se tuvó más volumenes tenía la palabra "Actualmente", y para inferir para que año refiere debía extaer información sobre la fecha de publicación al principio del documento.
Esta fue una preguntada pensada para evaluar la capacidad de recuperación de contexto de los splitters.

La pregunta referida al año del final del mandato como director de Acuña de Figueroa también implicaba cierto "razonamiento", ya que se menciona un año de comienzo en un chunk, y una duración del mandato en otro chunk.
Al analizar el contexto obtenido, nuevamente se observa que los títulos del markdown agregan ruido. El chunk que contenía la duración del mandato
no fue recuperado, y con la información recuperada (varias fechas de momentos relacionados a Figueroa) no era posible inferir el año final del mandato.

Finalmente, la pregunta sobre la ubicación de la Biblioteca fue respondida "correctamente" por dos de los tres experimentos ya que en ambos se respondió "No, ", pero los detalles provistos a continuación son inexactos e incompletos.
De estas observaciones surge la cuestión de qué significa que una pregunta sea respondida correctamente, por lo cuál a algunas respuestas le otorgamos medio punto de correctitud, marcado con un * en la tabla de resultados.

Tal vez se precise un mayor procesamiento de los textos y del contexto obtenido como más relevante para cada pregunta.

Hubo una pregunta particular que todos los experimentos fallaron al responderla (arquitecto /  Luis Crespi). Al observar cuáles habían sido el contexto recuperado para la misma, se observó que en al usar el RecursiveSplitter se recuperaban chunks relacionados a la pregunta pero insuficientes para inferir la respuesta, mientras que para el MarkdownSplitter los chunks obtenidos directamente no estaban relacionados a la pregunta, debido a que la información extra añadida por los títulos terminaba "contaminando" o agregando ruido a la búsqueda de contexto.

Las preguntas no relacionadas fueron respondidas correctamente como fuera de dominio por los tres experimentos. Considerando que el RAG no pudo obtener o encontrar respuestas en el texto a preguntas dentro del dominio, no es un indicador demasiado significativo. Por lo menos es rescatable el hecho de que se respeto la directiva de utilizar solo la información del contexto, aunque seguro era información que se encontraba en su memoria paramétrica.  

